# Semantic Parsing Final Project
Link to the paper: https://aclanthology.org/P16-1004.pdf

Read through the paper fully before starting the assignment!

In [1]:
import torch
import torch.nn as nn

from google.colab import drive
drive.mount('/content/drive')

FILEPATH = "drive/MyDrive/"

Mounted at /content/drive


# Data Downloading
This cell obtains the pre-processed Jobs dataset (see the paper) that you will be using to train and evaluate your model. (Pre-processed meaning that argument identification, section 3.6, has already been done for you). You should only need to run this cell ***once***. Feel free to delete it after running. Create a folder in your Google Drive in which the code below will store the pre-processed data needed for this project. Modify `FILEPATH` above to direct to said folder. It should start with `drive/MyDrive/...`, feel free to take a look at previous assignments that use mounting Google Drive if you can't remember what it should look like. *Make sure the data path ends with a slash character ('/').* The below code will access the zip file containing the pre-processed Jobs dataset from the paper and extract the files into your folder! Feel free to take a look at the `train.txt` and `test.txt` files to see what the data looks like. :)

In [2]:
'''
import requests
import io
import zipfile

# https://stackoverflow.com/questions/31126596/saving-response-from-requests-to-file
response = requests.get('http://dong.li/lang2logic/seq2seq_jobqueries.zip')
if response.status_code == 200:
  # https://stackoverflow.com/questions/3451111/unzipping-files-in-python
  with zipfile.ZipFile(io.BytesIO(response.content), "r") as zip_ref:
    zip_ref.extractall(FILEPATH)
  print("Extraction completed.")
else:
  print("Failed to download the zip file.")
  '''

'\nimport requests\nimport io\nimport zipfile\n\n# https://stackoverflow.com/questions/31126596/saving-response-from-requests-to-file\nresponse = requests.get(\'http://dong.li/lang2logic/seq2seq_jobqueries.zip\')\nif response.status_code == 200:\n  # https://stackoverflow.com/questions/3451111/unzipping-files-in-python\n  with zipfile.ZipFile(io.BytesIO(response.content), "r") as zip_ref:\n    zip_ref.extractall(FILEPATH)\n  print("Extraction completed.")\nelse:\n  print("Failed to download the zip file.")\n  '

# Data Pre-processing
The following code is defined for you! It extracts the queries (inputs to your Seq2Seq model) and logical forms (expected outputs) from the training and testing files. It also does important pre-processing such as padding the queries and logical forms and turns the words into vocab indices. **Look over and understand this code before you start the assignment!**

In [3]:
def extract_file(filename):
  """
  Extracts queries and corresponding logical forms from either
  train.txt or test.txt. (Feel free to take a look at the files themselves
  in your Drive!)

  Parameters
  ----------
  filename : str
      name of the file to extract from

  Returns
  ----------
  tuple[list[list[str]], list[list[str]]]
      a tuple of a list of queries and their corresponding logical forms
      each in the form of a list of string tokens
  """
  queries, logical_forms = [], []
  with open(FILEPATH + filename) as f:
    for line in f:
      line = line.strip() # remove new line character
      query, logical_form = line.split('\t')

      query = query.split(' ')[::-1] # reversed inputs are used the paper (section 4.2)
      logical_form = ["<s>"] + logical_form.split(' ') + ["</s>"]

      queries.append(query)
      logical_forms.append(logical_form)
  return queries, logical_forms

query_train, lf_train = extract_file('train.txt') # 500 instances
query_test, lf_test = extract_file('test.txt') # 140 instances

In [4]:
from collections import Counter

query_vocab = Counter()
for l in query_train:
  query_vocab.update(l)

query_word2idx = {}
for w, c in query_vocab.items():
  if c >= 2:
    query_word2idx[w] = len(query_word2idx)
query_word2idx['<UNK>'] = len(query_word2idx)
query_word2idx['<PAD>'] = len(query_word2idx)
query_idx2word = {i:word for word,i in query_word2idx.items()}

query_vocab = list(query_word2idx.keys())

lf_vocab = Counter()
for lf in lf_train:
  lf_vocab.update(lf)

lf_vocab['<UNK>'] = 0
lf_vocab['<PAD>'] = 0
lf_idx2word = {i:word for i, word in enumerate(lf_vocab.keys())}
lf_word2idx = {word:i for i, word in lf_idx2word.items()}

In [5]:
query_train_tokens = [[query_word2idx.get(w, query_word2idx['<UNK>']) for w in l] for l in query_train]
query_test_tokens = [[query_word2idx.get(w, query_word2idx['<UNK>']) for w in l] for l in query_test]

lf_train_tokens = [[lf_word2idx.get(w, lf_word2idx['<UNK>']) for w in l] for l in lf_train]
lf_test_tokens = [[lf_word2idx.get(w, lf_word2idx['<UNK>']) for w in l] for l in lf_test]

def pad(seq, max_len, pad_token_idx):
  """
  Pads a given sequence to the max length using the given padding token index

  Parameters
  ----------
  seq : list[int]
      sequence in the form of a list of vocab indices
  max_len : int
      length sequence should be padded to
  pad_token_idx
      vocabulary index of the padding token

  Returns
  ----------
  list[int]
      padded sequence
  """
  seq = seq[:max_len]
  padded_seq = seq + (max_len - len(seq)) * [pad_token_idx]
  return padded_seq

query_max_target_len = max([len(i) for i in query_train_tokens])
query_train_tokens = [pad(i, query_max_target_len, query_word2idx['<PAD>']) for i in query_train_tokens]
query_test_tokens = [pad(i, query_max_target_len, query_word2idx['<PAD>']) for i in query_test_tokens]

lf_max_target_len = int(max([len(i) for i in lf_train_tokens]) * 1.5)
lf_train_tokens = [pad(i, lf_max_target_len, lf_word2idx['<PAD>']) for i in lf_train_tokens]
lf_test_tokens = [pad(i, lf_max_target_len, lf_word2idx['<PAD>']) for i in lf_test_tokens]


# Data Loading
The following code creates a JobsDataset and DataLoaders to use with your implemented model. Take a look at the main function at the end of this stencil to see how they are used in context.

In [6]:
from torch.utils.data import Dataset, DataLoader, default_collate

class JobsDataset(Dataset):

  '''
  Defines a Dataset object for the Jobs dataset to be used with Dataloader
  '''

  def __init__(self, queries, logical_forms):
    """
    Initializes a JobsDataset

    Parameters
    ----------
    queries : list[list[int]]
        a list of queries, which have been tokenized and padded, in the form
        of a list of vocab indices
    logical_forms : list[list[int]]
        a list of corresponding logical forms, which have been tokenized and
        padded, in the form of a list of vocab indices
    """
    self.queries = queries
    self.logical_forms = logical_forms

  def __len__(self) -> int:
    """
    Returns the amount of paired queries and logical forms in the dataset

    Returns
    ----------
    int
        length of the dataset
    """
    return len(self.queries)

  def __getitem__(self, idx: int) -> tuple[list[int], list[int]]:
    """
    Returns a paired query and logical form at the specified index

    Parameters
    ----------
    idx : int
        specified index of the dataset

    Returns
    ----------
    tuple[list[int], list[int]]
        paired query and logical form at the specified index, in the form of
        a list of vocab indices
    """
    return self.queries[idx], self.logical_forms[idx]

def build_datasets() -> tuple[JobsDataset, JobsDataset]:
  """
  Builds a train and a test dataset from the queries and logical forms
  train and test tokens

  Returns
  ----------
  tuple[JobsDataset, JobsDataset]
      a training and testing JobsDataset
  """
  jobs_train = JobsDataset(queries=query_train_tokens, logical_forms=lf_train_tokens)
  jobs_test = JobsDataset(queries=query_test_tokens, logical_forms=lf_test_tokens)
  return jobs_train, jobs_test

def collate(batch : list[tuple[list[int], list[int]]]) -> tuple[torch.Tensor, torch.Tensor]:
  """
  Used as collate_fn when creating the Dataloaders from the dataset

  Parameters
  ----------
  batch : list[tuple[list[int], list[int]]]
      a list of outputs of __getitem__

  Returns
  ----------
  tuple[torch.Tensor, torch.Tensor]
      a batched set of input sequences and a batched set of target sequences
  """
  src, tgt = default_collate(batch)
  return torch.stack(src), torch.stack(tgt)

def build_dataloaders(dataset_train: JobsDataset, dataset_test: JobsDataset,
                      train_batch_size: int) -> tuple[DataLoader, DataLoader]:
  """
  Used as collate_fn when creating the Dataloaders from the dataset, batching
  the training data according to the inputted batch size and batching the
  testing data with a batch size of 1

  Parameters
  ----------
  dataset_train : JobsDataset
      training dataset
  dataset_test : JobsDataset
      testing dataset
  train_batch_size : int
      batch size to be used during training

  Returns
  ----------
  tuple[DataLoader, DataLoader]
      a training and testing DataLoader
  """
  dataloader_train = DataLoader(dataset_train, batch_size=train_batch_size, shuffle=True, collate_fn=collate)
  dataloader_test = DataLoader(dataset_test, batch_size=1, shuffle=False, collate_fn=collate)
  return dataloader_train, dataloader_test

# TODO: Define your model here!

In [7]:
from torch.nn import LSTM

class Encoder(nn.Module):
    '''
    Defines the Encoder module
    This module encodes input sequences into vector representations from which the logical forms can be decoded
    '''

    def __init__(self, input_size: int, num_layers: int):
      '''
      Initializes the Encoder module

      Parameters
      ----------
      input_size : int
          The size of the feature vector for each token in the input sequence.
      num_layers : int
          The number of LSTM layers in the encoder.
      '''
      super(Encoder, self).__init__()
      self.lstm = LSTM(input_size, input_size, num_layers=num_layers, batch_first=True)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
      '''
      Performs the encoder foward pass

      Parameters
      ----------
      x : torch.Tensor
          Input sequence as a tensor of token indices

      Returns
      -------
      tuple[torch.Tensor, tuple[torch.Tensor, torch.Tensor]]
          The output of the LSTM, which includes the output tensor and the hidden states
      '''
      return self.lstm(x)

In [8]:
import torch.nn.functional as F

class Decoder(nn.Module):
  '''
  Defines the Decoder module
  This module generates logical forms from input sequences by conditioning its output on the encoded input sequence
  It also integrates an attention mechanism to focus on relevant parts of the input during decoding, enhancing the accuracy of logical form generation.
  '''

  def __init__(self, input_size: int, num_layers: int):
      '''
      Initializes the Decoder module

      Parameters
      ----------
      input_size : int
          The size of the feature vector for each token in the sequence
      num_layers : int
          The number of LSTM layers in the decoder
      '''
      super(Decoder, self).__init__()
      self.lstm = LSTM(input_size, input_size, num_layers=num_layers, batch_first=True)


  def compute_attn(self, encoder_output: torch.Tensor, decoder_output: torch.Tensor) -> torch.Tensor:

      '''
      Computes the attention weights and context vectors from the encoder output to guide the decoding process.

      Parameters
      ----------
      encoder_output : torch.Tensor
          The output of the encoder, which contains the hidden states for each token.
      decoder_output : torch.Tensor
          The current decoder output

      Returns
      ----------
      torch.Tensor
          The updated decoder hidden state, which incorporates the context from the encoder.
      '''

      # Compute dot products between decoder and encoder hidden states (e_ht  * d_ht)
      dot_products = torch.matmul(decoder_output, encoder_output.transpose(1, 2))

      # Compute exp(dot_products) and normalize across encoder time steps. This is s_k^t in the paper
      attention_weights = F.softmax(dot_products, dim=-1)

      # Compute context vectors (ct)
      context_vectors = torch.matmul(attention_weights, encoder_output)

      # Combine decoder hidden states with context vectors to get update decoder hidden state
      ht_attn = torch.tanh(decoder_output + context_vectors)

      return ht_attn


  def forward(self, x: torch.Tensor, e_out: tuple[torch.Tensor, tuple[torch.Tensor, torch.Tensor]]) -> torch.Tensor:

    '''
    Performs the decoder forward pass

    Parameters
    ----------
    x : torch.Tensor
        The input sequence (model's target sequence)
    e_out : tuple[torch.Tensor, tuple[torch.Tensor, torch.Tensor]]
        The encoder's output, including the encoder's hidden states

    Returns
    ----------
    torch.Tensor
        The updated decoder hidden state after attending to the encoder output.
    '''

    encoder_output, (h_0, c_0) = e_out
    decoder_output, _ = self.lstm(x, (h_0, c_0))
    ht_attn = self.compute_attn(encoder_output, decoder_output)
    return ht_attn

In [9]:
import math

class TokenEmbedding(nn.Module):
  def __init__(self, vocab_size: int, emb_size: int, padding_idx):
      """
      Initializes a TokenEmbedding module

      Parameters
      ----------
      vocab_size : int
          size of the vocab that embeddings will be from
      emb_size : int
          embedding size
      padding_idx : int
          index of the padding token in the vocab
      """
      super(TokenEmbedding, self).__init__()
      self.embedding = nn.Embedding(vocab_size, emb_size, padding_idx=padding_idx)
      self.emb_size = emb_size

  def forward(self, tokens: torch.Tensor):
      """
      Returns the embedding of the tokens multiplied by sqrt(emb_size)

      Parameters
      ----------
      tokens : torch.Tensor
          input Tensor in the form of batched tokenized sentences using
          vocabulary indices

      Returns
      ----------
      torch.Tensor
          output of the embeddings
      """
      return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

In [10]:
class Seq2Seq(nn.Module):
  '''
  Defines the Seq2Seq model
  The model maps input sequences to machine-interpretable logical forms.
  An encoder-decoder architecture with attention is used to generate the target sequence based on the encoded input sequence.
  '''

  def __init__(self,
               src_vocab_size: int,
               tgt_vocab_size: int,
               emb_size: int,
               num_layers: int):
    '''
    Initializes the Seq2Seq model

    Parameters
    ----------
     src_vocab_size : int
            The size of the source vocabulary.
    tgt_vocab_size : int
        The size of the target vocabulary.
    emb_size : int
        The embedding size used for token embeddings in both the encoder and decoder.
    num_layers : int
        The number of LSTM layers in both the encoder and decoder.
    '''
    super(Seq2Seq, self).__init__()
    lf_pad_index = lf_word2idx['<PAD>']
    query_pad_index = query_word2idx['<PAD>']
    self.out = nn.Linear(emb_size, tgt_vocab_size) #to project back from embeddings to tgt vocab size
    self.in_emb = TokenEmbedding(src_vocab_size, emb_size, padding_idx=query_pad_index)
    self.out_emb = TokenEmbedding(tgt_vocab_size, emb_size, padding_idx=lf_pad_index)
    self.encoder = Encoder(emb_size, num_layers)
    self.decoder = Decoder(emb_size, num_layers)

  def encode(self, src: torch.Tensor) -> torch.Tensor:
    '''
    Performs the full encoding pass

    Parameters
    ----------
    src : torch.Tensor
        The input sequence to be encoded

    Returns
    ----------
    torch.Tensor
        The output of the encoding pass
    '''
    return self.encoder(src)


  def decode(self, tgt: torch.Tensor, e_out: tuple[torch.Tensor, tuple[torch.Tensor, torch.Tensor]]) -> torch.Tensor:
    '''
    Performs the full decoding pass, conditioned on the encoder output

    Parameters
    ----------
    tgt : torch.Tensor
        The model's target sequence
    e_out : tuple[torch.Tensor, tuple[torch.Tensor, torch.Tensor]]
        The output of the encoder

    Returns
    ----------
    torch.Tensor
        The output of the forward pass
    '''
    return self.decoder(tgt, e_out)


  def forward(self, src: torch.Tensor, tgt: torch.Tensor) -> torch.Tensor:
    '''
    Performs the full Seq2Seq foward pass

    Parameters
    ----------
    src : torch.Tensor
        The model's input sequence
    tgt : torch.Tensor
        The model's target sequence

    Returns
    ----------
    torch.Tensor
        The output of the forward pass
    '''
    src = self.in_emb(src)
    tgt = self.out_emb(tgt)
    encoder_output = self.encode(src)
    output = self.decode(tgt, encoder_output)
    log_probs = F.log_softmax(self.out(output), dim=-1)
    return log_probs


In [11]:
QUERY_VOCAB_LEN = len(query_vocab)
LF_VOCAB_LEN = len(lf_vocab)

def create_model():
  """
  Create and returns the Seq2Seq model!

  Returns
  ----------
  The Seq2Seq model
  """
  num_layers = 2
  embedding_size = 1024
  model = Seq2Seq(QUERY_VOCAB_LEN, LF_VOCAB_LEN, embedding_size, num_layers)
  return model

# TODO: Training and testing loops

In [12]:
LF_SOS_INDEX = lf_word2idx['<s>']
LF_EOS_INDEX = lf_word2idx['</s>']
LF_PAD_INDEX = lf_word2idx['<PAD>']

In [13]:
def train_epoch(model: nn.Module,
                train_dataloader: DataLoader,
                loss_fn: nn.Module,
                optimizer: torch.optim.Optimizer,
                device: str="cuda") -> float:
    """
    Trains the model using the provided data, optimizer, and loss
    function for one epoch. Returns the average loss of the epoch.

    Parameters
    ----------
    model : nn.Module
        model to train
    optimizer : torch.optim.Optimizer
        optimizer to use with training
    train_dataloader : DataLoader
        training data
    loss_fn : nn.Module
        loss function to use with training
     device : str
      device that the model is running on

    Returns
    ----------
    float
        epoch average loss
    """
    model.train()
    total_loss = 0

    for src, tgt in train_dataloader:
        src = src.to(device)
        tgt = tgt.to(device)
        src = src.transpose(0, 1)
        tgt = tgt.transpose(0, 1)
        tgt_input = tgt[:, :-1]  # Input to decoder is all tokens except the last
        tgt_expected = tgt[:, 1:]  # Expected output is all tokens except the first

        # Forward pass
        logits = model(src, tgt_input) # Shape: (batch_size, seq_length, tgt_vocab_size)
        optimizer.zero_grad()
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_expected.reshape(-1))

        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    return total_loss / len(train_dataloader)

In [14]:
from torch.optim import Adam
from torch.nn import NLLLoss
from timeit import default_timer as timer

def train(model: nn.Module, train_dataloader: DataLoader, num_epochs: int=5,
          device: str="cuda") -> nn.Module:
  """
  Trains the Seq2Seq model!

  Parameters
  ----------
  model : nn.Module
      The untrained model
  train_dataloader : DataLoader
      a dataloader of the training data from build_dataloaders
  num_epochs : int
      number of epochs to train for
  device : str
      device that the model is running on

  Returns
  ----------
  the trained model
  """
  loss_fn = NLLLoss(ignore_index=LF_PAD_INDEX)
  optimizer = Adam(model.parameters(), lr=0.001)
  model = model.to(device)

  for epoch in range(1, num_epochs+1):
      print(f"Epoch {epoch},", end = " ")
      start_time = timer()
      train_loss = train_epoch(model, train_dataloader, loss_fn, optimizer, device=device)
      end_time = timer()
      print(f" Train loss: {train_loss:.3f}, Epoch time = {(end_time - start_time):.3f}s")

  return model


In [15]:
def greedy_decode(model: nn.Module,
                  src: torch.Tensor,
                  max_len: int,
                  device: str = "cuda"
                  ) -> tuple[torch.Tensor, float]:
    """
    Performs greedy decoding to generate the logical form

    Parameters
    ----------
    model : nn.Module
        Trained model to decode from.
    src : torch.Tensor
        Source sequence inputted to the model
    max_len : int
        maximum length to generate with decoding
    device : str
        Device to run the model on (default is "cuda").

    Returns
    -------
    tuple[torch.Tensor, float]
        A tuple containing:
        - The generated sequence as a tensor of vocabulary indices.
        - The total log probability of the generated sequence.
    """
    model.eval()

     # Encode the source sequence
    src_embedding = model.in_emb(src)
    src_encoding  = model.encode(src_embedding)

    # Initialize the decoder input with the SOS token
    seq = torch.ones((1, 1), dtype=torch.long, device=device).fill_(LF_SOS_INDEX)
    total_prob = 0.0

    for _ in range(max_len - 1):
        seq_embedded = model.out_emb(seq)
        decoder_output = model.decode(seq_embedded, src_encoding)
        logits = model.out(decoder_output[:, -1, :]) #use h_out from last time step
        probs = F.log_softmax(logits, dim=-1)
        prob, next_word = torch.max(probs, dim=-1)
        total_prob += prob.item()
        next_word = next_word.item()

        # Append the next word to the sequence
        next_word_tensor = torch.ones((1, 1), dtype=torch.long, device=device).fill_(next_word)
        seq = torch.cat([seq, next_word_tensor], dim=1)

        # Stop decoding if the eos token is generated
        if next_word == LF_EOS_INDEX:
            break

    return seq, total_prob

In [16]:

def debug_seqs(tgt_seq, pred_seq):
  '''
  Prints both the ground truth logical form as well as the predicted one
  Used for debugging the sequences 😀
  '''
  trg_sentence = " ".join([lf_idx2word[idx.item()] for idx in tgt_seq if idx.item() in lf_idx2word])
  predicted_sentence = " ".join([lf_idx2word.get(idx.item(), '<UNK>') for idx in pred_seq])

  print(f"\nPrediction: {predicted_sentence}")
  print(f"\nTarget: {trg_sentence}\n")

In [17]:
from tqdm import tqdm

def evaluate(model: nn.Module, dataloader: DataLoader, device: str = "cuda") -> tuple[int, int]:
  """
  Evaluates the model using greedy decoding!

  Parameters
  ----------
  model : nn.Module
      Trained Seq2Seq model.
  dataloader : DataLoader
      A dataloader of the testing data from build_dataloaders.
  device : str
      Device that the model is running on (default: "cuda")

  Returns
  ----------
  tuple[int, int]
      Per-token accuracy and exact-match accuracy.
  """
  model.eval()
  total_tokens = 0
  correct_tokens = 0
  total_sequences = 0
  correct_sequences = 0

  with torch.no_grad():
      for batch in tqdm(dataloader, desc="Evaluating", unit="batch"):
          src, tgt = batch
          src, tgt = src.transpose(0,1).to(device), tgt.transpose(0, 1).to(device)

          # Perform greedy decoding
          max_len = torch.where(tgt == LF_EOS_INDEX)[1][0] + 1
          decoded_seq, _ = greedy_decode(model, src, max_len=max_len, device=device)  # Use actual target length

           # Loop through each sequence in the batch
          for i in range(src.size(0)):
              predicted = decoded_seq[i].tolist()
              target = tgt[i].tolist()

              # Truncate to the first `max_len` tokens
              predicted = predicted[:max_len]
              predicted[-1] = LF_EOS_INDEX  # Ensure EOS token is present
              target = target[:max_len]

              # Token-level accuracy
              correct_tokens += sum(p == t for p, t in zip(predicted, target))
              total_tokens += len(target)

              # Sequence-level (exact match) accuracy
              if predicted == target:
                  correct_sequences += 1
              total_sequences += 1

  # Calculate accuracy metrics
  per_token_accuracy = correct_tokens / total_tokens if total_tokens > 0 else 0
  exact_match_accuracy = correct_sequences / total_sequences if total_sequences > 0 else 0

  return per_token_accuracy, exact_match_accuracy

# Run this!

In [18]:
def main():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    n_epochs = 20
    jobs_train, jobs_test = build_datasets()
    dataloader_train, dataloader_test = build_dataloaders(jobs_train, jobs_test, train_batch_size=20)
    model = create_model()
    model = train(model, dataloader_train, num_epochs=n_epochs, device=device)
    test_per_token_accuracy, test_exact_match_accuracy = evaluate(model, dataloader_test, device=device)
    print(f'\nTest Per-token Accuracy: {test_per_token_accuracy}')
    print(f'Test Exact-match Accuracy: {test_exact_match_accuracy}')
main()

Epoch 1,  Train loss: 1.264, Epoch time = 3.291s
Epoch 2,  Train loss: 0.450, Epoch time = 1.982s
Epoch 3,  Train loss: 0.284, Epoch time = 1.985s
Epoch 4,  Train loss: 0.220, Epoch time = 2.008s
Epoch 5,  Train loss: 0.181, Epoch time = 2.023s
Epoch 6,  Train loss: 0.134, Epoch time = 2.028s
Epoch 7,  Train loss: 0.108, Epoch time = 2.029s
Epoch 8,  Train loss: 0.090, Epoch time = 2.035s
Epoch 9,  Train loss: 0.091, Epoch time = 2.067s
Epoch 10,  Train loss: 0.064, Epoch time = 2.052s
Epoch 11,  Train loss: 0.048, Epoch time = 2.044s
Epoch 12,  Train loss: 0.043, Epoch time = 2.069s
Epoch 13,  Train loss: 0.043, Epoch time = 2.070s
Epoch 14,  Train loss: 0.046, Epoch time = 2.089s
Epoch 15,  Train loss: 0.052, Epoch time = 2.107s
Epoch 16,  Train loss: 0.033, Epoch time = 2.126s
Epoch 17,  Train loss: 0.030, Epoch time = 2.141s
Epoch 18,  Train loss: 0.032, Epoch time = 2.153s
Epoch 19,  Train loss: 0.024, Epoch time = 2.173s
Epoch 20,  Train loss: 0.029, Epoch time = 2.197s


Evaluating: 100%|██████████| 140/140 [00:07<00:00, 19.24batch/s]


Test Per-token Accuracy: 0.8788213627992634
Test Exact-match Accuracy: 0.8214285714285714
